In [15]:
!pip install scikeras

In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import os

In [17]:
try:
    previsores = pd.read_csv('dados_breast.csv', header=None)
    classes = pd.read_csv('rotulos_breast.csv', header=None)
except FileNotFoundError:
    print("Erro: Arquivos 'dados_breast.csv' ou 'rotulos_breast.csv' não encontrados.")
    print("Por favor, coloque os arquivos de dados no mesmo diretório do script.")
    exit()

In [18]:
print("--- Amostra dos Atributos (Previsores) ---")
print(previsores.head())
print("\n--- Amostra das Classes (Rótulos) ---")
print(classes.head())

--- Amostra dos Atributos (Previsores) ---
             0              1                2           3                 4   \
0   radius_mean   texture_mean   perimeter_mean   area_mean   smoothness_mean   
1         17.99          10.38            122.8        1001            0.1184   
2         20.57          17.77            132.9        1326           0.08474   
3         19.69          21.25              130        1203            0.1096   
4         11.42          20.38            77.58       386.1            0.1425   

                  5                6                    7               8   \
0   compactness_mean   concavity_mean  concave_points_mean   symmetry_mean   
1             0.2776           0.3001               0.1471          0.2419   
2            0.07864           0.0869              0.07017          0.1812   
3             0.1599           0.1974               0.1279          0.2069   
4             0.2839           0.2414               0.1052          0.2597   

 

In [19]:
# 1) Quantos exemplos e atributos a base de dados possui?
print(f"\nDimensões da base de dados (exemplos, atributos): {previsores.shape}\n")


Dimensões da base de dados (exemplos, atributos): (570, 30)



In [20]:
previsores_treinamento, previsores_teste, classes_treinamento, classes_teste = train_test_split(previsores, classes, test_size=0.25, random_state=0)
print("Dados divididos em conjuntos de treinamento e teste.")

Dados divididos em conjuntos de treinamento e teste.


In [21]:
# 2) A RNA foi criada com quantas camadas? Quantos neurônios em cada? Qual a função de ativação de cada uma?
# Resposta: 3 camadas.
# Primeira camada oculta: 16 neurônios, função de ativação ReLU (Rectified Linear Unit)
# Segunda camada oculta: 16 neurônios, função de ativação ReLU
# Terceira camada de saída: 1 neurônio, função de ativação sigmoide

# 3) Por que a camada de saída possui apenas 1 neurônio?
# A camada de saída possui apenas 1 neurônio porque se trata de um problema de
# classificação binária, onde a saída desejada é uma única probabilidade entre
# duas classes, 0 e 1. Um único neurônio é suficiente para representar essa
# probabilidade.

# 4) Por que a função de ativação da camada de saída é a Sigmoid?
# Respostas: Ver relatório final. A arquitetura está definida abaixo.
classificador = keras.Sequential([
    tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30),
    tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
def imprimir_pesos(modelo, titulo):
    print(f"\n--- {titulo} ---")
    pesos = modelo.get_weights()
    for i, camada_pesos in enumerate(pesos):
        print(f"Formato da Camada {i}: {camada_pesos.shape}")
    print("--------------------------------------\n")

In [23]:
# 5) Os pesos da RNA são inicializados com quais valores?
imprimir_pesos(classificador, "Pesos Antes do Treinamento (Valores Iniciais)")


--- Pesos Antes do Treinamento (Valores Iniciais) ---
Formato da Camada 0: (30, 16)
Formato da Camada 1: (16,)
Formato da Camada 2: (16, 16)
Formato da Camada 3: (16,)
Formato da Camada 4: (16, 1)
Formato da Camada 5: (1,)
--------------------------------------



In [24]:
# 6) O que é o otimizador e a função de perda? Por que 'binary_crossentropy'?
classificador.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

# Resposta: O otimizador é um algoritmo responsável por ajustar os pesos da rede
# neural durante o treinamento, minimizando o erro (função de perda). O otimizador
# Adam (Adaptative Moment Estimation) é robusto, funciona bem na prática em muitos
# tipos de problemas e requer pouca configuração manual de hiperparâmetros.

# 'binary_crossentropy' é a função de perda que mede o erro entre a saída prevista
# pela rede e o valor real (rótulo). Ela é o que a rede tenta minimizar durante o
# treinamento. Nesse caso, ela foi escolhida por ser a mais apropriada para
# problemas de classificação binária (entropia cruzada binária).

In [25]:
print("Iniciando o treinamento do modelo...")
historico = classificador.fit(previsores_treinamento, classes_treinamento,
                            batch_size=10, epochs=100,
                            validation_data=(previsores_teste, classes_teste),
                            verbose=1) # verbose=1 para ver o progresso
print("Treinamento concluído.")

Iniciando o treinamento do modelo...


ValueError: Invalid dtype: object

In [ ]:
# 7) Compare os pesos antes e depois do treinamento. O que aconteceu?
imprimir_pesos(classificador, "Pesos Após o Treinamento")

In [ ]:
# 8) Os valores de acurácia e perda da base de treinamento melhoraram ao longo das épocas?
print("\nPlotando gráficos de acurácia e perda...")
plt.figure(figsize=(12, 5))

In [ ]:
plt.subplot(1, 2, 1)
plt.plot(historico.history['binary_accuracy'])
plt.plot(historico.history['val_binary_accuracy'])
plt.title('Histórico de Acurácia')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.legend(['Treinamento', 'Validação'], loc='upper left')

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(historico.history['loss'])
plt.plot(historico.history['val_loss'])
plt.title('Histórico de Perda')
plt.ylabel('Perda')
plt.xlabel('Época')
plt.legend(['Treinamento', 'Validação'], loc='upper right')

In [ ]:
plt.tight_layout()
plt.show()

In [ ]:
# 9) Qual foi o resultado da acurácia na base de teste?
print("\nAvaliando o modelo no conjunto de teste...")
resultado_teste = classificador.evaluate(previsores_teste, classes_teste, verbose=0)
print(f"Perda no teste: {resultado_teste[0]:.4f}")
print(f"Acurácia no teste: {resultado_teste[1]:.4f}")

In [ ]:
# 10) Qual o resultado da matriz de confusão na base de teste?
previsoes = classificador.predict(previsores_teste)
previsoes_binarias = (previsoes > 0.5)
matriz = confusion_matrix(classes_teste, previsoes_binarias)
print("\nMatriz de Confusão:")
print(matriz)

In [ ]:
# 11) Por que usar a Validação Cruzada?
# 12) O que faz o 'KerasClassifier'?

def criar_rna_cv():
    modelo = keras.Sequential([
        tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30),
        tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
    return modelo

In [ ]:
print("\nIniciando a Validação Cruzada (10 folds)...")
classificador_cv = KerasClassifier(build_fn=criar_rna_cv, epochs=100, batch_size=10, verbose=0)

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=0)

In [ ]:
resultados_cv = cross_val_score(classificador_cv, previsores, classes.values.ravel(), cv=kfold)
print("Validação Cruzada concluída.")

In [ ]:
# 13) Quais os resultados da validação cruzada?
print(f"\nResultados (acurácia) de cada fold: {np.round(resultados_cv, 4)}")

In [ ]:
# 14) Qual a média e o desvio padrão dos resultados?
print(f"Média das acurácias: {resultados_cv.mean():.4f}")
print(f"Desvio padrão das acurácias: {resultados_cv.std():.4f}")

In [ ]:
def criar_rna_tuning(optimizer='adam', loss='binary_crossentropy', kernel_initializer='random_uniform', activation='relu', neurons=16):
    modelo = keras.Sequential([
        tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer, input_dim=30),
        tf.keras.layers.Dense(units=neurons, activation=activation, kernel_initializer=kernel_initializer),
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    modelo.compile(optimizer=optimizer, loss=loss, metrics=['binary_accuracy'])
    return modelo

classificador_gs = KerasClassifier(build_fn=criar_rna_tuning, verbose=0)

In [ ]:
parametros = {
    'batch_size': [10, 30],
    'epochs': [50, 100],
    'optimizer': ['adam', 'sgd'],
    'loss': ['binary_crossentropy', 'hinge'],
    'kernel_initializer': ['random_uniform', 'normal'],
    'activation': ['relu', 'tanh'],
    'neurons': [16, 8]
}

In [ ]:
print("\nIniciando a Otimização com GridSearchCV (pode levar vários minutos)...")
grid_search = GridSearchCV(estimator=classificador_gs, param_grid=parametros, scoring='accuracy', cv=5)
grid_search.fit(previsores, classes.values.ravel())
print("GridSearchCV concluído.")

In [ ]:
# 15) Quais os melhores parâmetros encontrados?
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [ ]:
print("\nMelhores parâmetros encontrados:")
print(melhores_parametros)
print(f"\nMelhor precisão encontrada (acurácia média): {melhor_precisao:.4f}")

In [ ]:
# 16) É possível melhorar ainda mais a acurácia da RNA? Como?
# Resposta: Sim, com engenharia de atributos, mais dados, busca de hiperparâmetros mais ampla, regularização, etc.

In [ ]:
print("\nSalvando o modelo treinado...")
caminho_modelo = 'classificador_breast.keras'
classificador.save(caminho_modelo)
print(f"Modelo salvo em '{caminho_modelo}'")

In [ ]:
if os.path.exists(caminho_modelo):
    print("\nCarregando o modelo salvo...")
    classificador_novo = tf.keras.models.load_model(caminho_modelo)
    print("Modelo carregado com sucesso.")

    # Exemplo de predição com o modelo carregado
    novo_exemplo = previsores_teste.iloc[0:1]
    previsao_nova = classificador_novo.predict(novo_exemplo)
    print(f"\nPredição para um novo exemplo: {previsao_nova[0][0]:.4f}")
    print(f"Classe prevista: {'Maligno' if previsao_nova[0][0] > 0.5 else 'Benigno'}")
else:
    print(f"\nArquivo de modelo '{caminho_modelo}' não encontrado para carregar.")

print("\n--- FIM DO SCRIPT ---")